In [97]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# HuggingFace datasets and tokenizers
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from tdg_dataset import BilingualDataset

from config import get_config

from pathlib import Path

In [98]:
config = get_config()

In [99]:
config

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'datasource': 'opus_books',
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': 'latest',
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [100]:
# load raw dataset
ds_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

In [101]:
ds_raw

Dataset({
    features: ['id', 'translation'],
    num_rows: 32332
})

In [102]:
len(ds_raw['id'])

32332

In [103]:
type(ds_raw['translation'])

list

In [104]:
ds_raw['translation'][0]

{'en': 'Source: Project Gutenberg',
 'it': 'Source: www.liberliber.it/Audiobook available here'}

In [105]:

def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

In [106]:
def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    print(tokenizer_path)
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

In [107]:
# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

tokenizer_en.json
tokenizer_it.json


In [108]:
ds_raw

Dataset({
    features: ['id', 'translation'],
    num_rows: 32332
})

In [109]:
train_ds_size = int(0.9 * len(ds_raw))
val_ds_size = len(ds_raw) - train_ds_size
train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

In [110]:
train_ds_size

29098

In [111]:
val_ds_size

3234

In [112]:
config

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'datasource': 'opus_books',
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': 'latest',
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [113]:
for item in ds_raw:
    print(item['translation'][config['lang_src']])
    print(tokenizer_src.encode(item['translation'][config['lang_src']]))
    print(tokenizer_src.encode(item['translation'][config['lang_src']]).tokens)
    print(tokenizer_src.encode(item['translation'][config['lang_src']]).ids)
    break

Source: Project Gutenberg
Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['Source', ':', 'Project', '[UNK]']
[5781, 38, 7699, 0]


In [114]:
for item in ds_raw:
    print(item['translation'][config['lang_tgt']])
    print(tokenizer_tgt.encode(item['translation'][config['lang_tgt']]))
    print(tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).tokens)
    print(tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids)
    break

Source: www.liberliber.it/Audiobook available here
Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['Source', ':', '[UNK]', '.', 'liberliber', '.', 'it', '/', '[UNK]', 'available', 'here']
[8161, 43, 0, 5, 19606, 5, 19516, 10657, 0, 13463, 14295]


In [115]:
# Find the maximum length of each sentence in the source and target sentence
max_len_src = 0
max_len_tgt = 0

for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

print(f'Max length of source sentence: {max_len_src}')
print(f'Max length of target sentence: {max_len_tgt}')

Max length of source sentence: 309
Max length of target sentence: 274


In [116]:
config

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'datasource': 'opus_books',
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': 'latest',
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [117]:
config['seq_len']

350

In [118]:
config['batch_size']

8

In [119]:
train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

In [120]:
train_ds

In [121]:
torch.tensor(
            [tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64
        )

tensor([2])

In [122]:
tokenizer_tgt.token_to_id("[SOS]")

2

In [123]:
tokenizer_src.token_to_id("[SOS]")

2

In [124]:
tokenizer_tgt.token_to_id("[EOS]")

3

In [125]:
tokenizer_src.token_to_id("[EOS]")

3

In [126]:
tokenizer_src.token_to_id("[PAD]")

1

In [127]:
tokenizer_src.token_to_id("[UNK]")

0

In [128]:
ds_raw[0]

{'id': '0',
 'translation': {'en': 'Source: Project Gutenberg',
  'it': 'Source: www.liberliber.it/Audiobook available here'}}

In [129]:
ds_raw[100]

{'id': '100',
 'translation': {'en': 'Miss Abbot joined in-- "And you ought not to think yourself on an equality with the Misses Reed and Master Reed, because Missis kindly allows you to be brought up with them.',
  'it': 'La signorina Abbot soggiunse: — Spero che non vi crederete eguale alle signorine e al signor Reed, perché la signora è così buona da farvi educare insieme con loro.'}}

In [130]:
idx = 100
seg_len = config['seq_len']
src_target_pair = ds_raw[idx]


In [131]:
src_target_pair

{'id': '100',
 'translation': {'en': 'Miss Abbot joined in-- "And you ought not to think yourself on an equality with the Misses Reed and Master Reed, because Missis kindly allows you to be brought up with them.',
  'it': 'La signorina Abbot soggiunse: — Spero che non vi crederete eguale alle signorine e al signor Reed, perché la signora è così buona da farvi educare insieme con loro.'}}

In [132]:
src_text = src_target_pair["translation"][config['lang_src']]
tgt_text = src_target_pair["translation"][config['lang_tgt']]

# Transform the text into tokens
enc_input_tokens = tokenizer_src.encode(src_text).ids
dec_input_tokens = tokenizer_tgt.encode(tgt_text).ids

# Add SOS, EOS and padding to each sentence
enc_num_padding_token = (
    seg_len - len(enc_input_tokens) - 2
)  # We will add  <SOS> and <EOS> tokens
dec_num_padding_token = (
    seg_len - len(dec_input_tokens) - 1
)  # We will add <EOS> toke

In [133]:
len(dec_input_tokens)

32

In [134]:
tgt_text

'La signorina Abbot soggiunse: — Spero che non vi crederete eguale alle signorine e al signor Reed, perché la signora è così buona da farvi educare insieme con loro.'

In [135]:
sos_token = torch.tensor(
    [tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64
)
eos_token = torch.tensor(
    [tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64
)
pad_token = torch.tensor(
    [tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64
)

In [136]:
print(sos_token, eos_token, pad_token)

tensor([2]) tensor([3]) tensor([1])


In [137]:
enc_input_tokens

[317,
 2570,
 1535,
 13,
 78,
 35,
 91,
 24,
 388,
 21,
 8,
 153,
 544,
 36,
 65,
 8008,
 22,
 5,
 5762,
 698,
 6,
 3443,
 698,
 4,
 173,
 3324,
 1536,
 8843,
 24,
 8,
 37,
 291,
 56,
 22,
 50,
 7]

In [138]:
encoder_input = torch.cat(
        [
            sos_token,
            torch.tensor(enc_input_tokens, dtype=torch.int64),
            eos_token,
            torch.tensor(
                [pad_token] * enc_num_padding_token, dtype=torch.int64
            ),
        ],
        dim=0,
    )

In [139]:
encoder_input

tensor([   2,  317, 2570, 1535,   13,   78,   35,   91,   24,  388,   21,    8,
         153,  544,   36,   65, 8008,   22,    5, 5762,  698,    6, 3443,  698,
           4,  173, 3324, 1536, 8843,   24,    8,   37,  291,   56,   22,   50,
           7,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,   

In [140]:
decoder_input = torch.cat(
            [
                sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor(
                    [pad_token] * dec_num_padding_token, dtype=torch.int64
                ),
            ],
            dim=0,
        )

In [141]:
decoder_input

tensor([    2,    79,   357,  2496,   911,    43,     9,  2274,     8,    12,
           64, 18158,  2365,   162,  2568,     6,    39,   175,   746,     4,
           70,    11,   209,    27,    55,   400,    28,  2527,  7550,   347,
           20,    69,     5,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [142]:
label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                eos_token,
                torch.tensor(
                    [pad_token] * dec_num_padding_token, dtype=torch.int64
                ),
            ],
            dim=0,
        )

In [143]:
label

tensor([   79,   357,  2496,   911,    43,     9,  2274,     8,    12,    64,
        18158,  2365,   162,  2568,     6,    39,   175,   746,     4,    70,
           11,   209,    27,    55,   400,    28,  2527,  7550,   347,    20,
           69,     5,     3,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [144]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

In [145]:
output = {
            "encoder_input": encoder_input,  # Shape: (seq_len,)
            "decoder_input": decoder_input,  # Shape: (seq_len,)
            "encoder_mask": (encoder_input != pad_token)
            .unsqueeze(0)
            .unsqueeze(0)
            .int(),  # Shape: (1, 1, seq_len)
            "decoder_mask": (decoder_input != pad_token).unsqueeze(0).int()
            & causal_mask(
                decoder_input.shape[0]
            ),  # Shape: (1, seq_len) &  (1, seq_len, seq_len)
            "label": label,  # Shape: (seq_len,)
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

In [146]:
label.shape

torch.Size([350])

In [147]:
pad_token

tensor([1])

In [148]:
(encoder_input != pad_token).shape

torch.Size([350])

In [149]:
(encoder_input != pad_token)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [150]:
(encoder_input != pad_token).unsqueeze(0).unsqueeze(0).int()

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [151]:
(encoder_input != pad_token).unsqueeze(0).unsqueeze(0).int().shape

torch.Size([1, 1, 350])

In [152]:
(decoder_input != pad_token).unsqueeze(0).int()

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [159]:
(decoder_input != pad_token).unsqueeze(0).int().shape

torch.Size([1, 350])

In [158]:
decoder_input.shape[0]

350

In [154]:
causal_mask(decoder_input.shape[0])

tensor([[[ True, False, False,  ..., False, False, False],
         [ True,  True, False,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ...,  True, False, False],
         [ True,  True,  True,  ...,  True,  True, False],
         [ True,  True,  True,  ...,  True,  True,  True]]])

In [157]:
output['decoder_mask'].shape

torch.Size([1, 350, 350])

In [156]:
len(output['decoder_mask'])

1

In [83]:
import torch

def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

# Let's say we have the following decoder input and pad token
decoder_input = torch.tensor([1, 2, 3, 0, 0])  # 0 is the pad token
pad_token = 0

# Create a mask where True represents non-pad tokens
non_pad_mask = (decoder_input != pad_token).unsqueeze(0).int()

# Create a causal mask for the decoder input
causal_mask = causal_mask(decoder_input.shape[0])

# Combine the two masks
combined_mask = non_pad_mask & causal_mask

print(combined_mask)

tensor([[[1, 0, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0]]], dtype=torch.int32)


In [84]:
non_pad_mask

tensor([[1, 1, 1, 0, 0]], dtype=torch.int32)

In [85]:
causal_mask

tensor([[[ True, False, False, False, False],
         [ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True]]])

In [86]:
combined_mask

tensor([[[1, 0, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0]]], dtype=torch.int32)

In [87]:
output['encoder_mask']

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [88]:
output['decoder_mask']

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]], dtype=torch.int32)

In [89]:
print(output['encoder_mask'].shape)

torch.Size([1, 1, 350])


In [90]:
print(output['decoder_mask'].shape)

torch.Size([1, 350, 350])


In [93]:
train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=True)

In [94]:
train_dataloader

In [95]:
val_dataloader

In [96]:
train_ds

In [160]:
train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=True)

In [161]:
# Assuming train_dataloader is a DataLoader object
for i, batch in enumerate(train_dataloader):
    print(f"Batch {i+1}:")
    print(batch)
    # Only print the first 2 batches for brevity
    if i >= 1:
        break

Batch 1:
{'encoder_input': tensor([[   2,  240, 3674,  ...,    1,    1,    1],
        [   2,  337,   69,  ...,    1,    1,    1],
        [   2, 5781,   38,  ...,    1,    1,    1],
        ...,
        [   2,  117,  232,  ...,    1,    1,    1],
        [   2, 4446,  447,  ...,    1,    1,    1],
        [   2,   12,    9,  ...,    1,    1,    1]]), 'decoder_input': tensor([[    2,     0,   807,  ...,     1,     1,     1],
        [    2,   241,    42,  ...,     1,     1,     1],
        [    2,  8161,    43,  ...,     1,     1,     1],
        ...,
        [    2,     0,    45,  ...,     1,     1,     1],
        [    2, 16685,   291,  ...,     1,     1,     1],
        [    2,     9,   136,  ...,     1,     1,     1]]), 'encoder_mask': tensor([[[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        ...,


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]

In [163]:
type(batch)

dict

In [165]:
batch.keys()

dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'src_text', 'tgt_text'])

In [167]:
len(batch['encoder_input'][1])

350

In [168]:
batch['encoder_input'][1].shape

torch.Size([350])